In [ ]:
import pickle

# Load the TfidfVectorizer
with open('tfidf_vectorizer.pkl', 'rb') as file:
    tfidf = pickle.load(file)

# Load the transformed training data
with open('X_train_tfidf.pkl', 'rb') as file:
    X_train_tfidf = pickle.load(file)

# Load the transformed testing data
with open('X_test_tfidf.pkl', 'rb') as file:
    X_test_tfidf = pickle.load(file)

# Load the feature names
with open('features_names_tfidf.pkl', 'rb') as file:
    features_names_tfidf = pickle.load(file)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize the LDA model
lda = LatentDirichletAllocation(n_components=10, random_state=0)  # Set the number of topics

# Fit the LDA model using the training data
lda.fit(X_train_tfidf)

# Transform the training data to LDA topic-space
X_train_lda = lda.transform(X_train_tfidf)

# Transform the testing data to LDA topic-space
X_test_lda = lda.transform(X_test_tfidf)

In [ ]:
# Function to display the top words in each topic
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

# Display the top 10 words for each topic
display_topics(lda, features_names_tfidf, 10)
